## Introduction

The primary use of [BioFindr.jl][1] is for causal inference from genomics and transcriptomics data. In causal inference, cis-acting eQTLs are used as causal anchors or instrumental variables to orient the direction of causality between coexpressed genes, see the [Findr paper][5] for more details. As in [coexpression analysis](coexpression.qmd) and [association analysis](association.qmd), the significance of a causal effect is computed using a gene-specific background distribution. This is again achieved by modelling the distribution of test statistics between a given gene $A$ and all other genes $B$ as a [mixture distribution](https://en.wikipedia.org/wiki/Mixture_distribution) of real and null (random) correlations. The relative weight of each component reflects the prior probability of finding a non-null $B$ gene for a given $A$ gene, and is fitted for every $A$ gene separately.

Unlike in [coexpression analysis](coexpression.qmd) and [association analysis](association.qmd), causal inference cannot be performed using a single statistical test, but requires the combination of multiple tests. In [BioFindr.jl][1], [six likelihood ratio tests](https://tmichoel.github.io/BioFindr.jl/dev/realLLR/) are implemented, which can be combined in multiple ways for causal inference. Tests are combined by addition or mulitplication of the [posterior probabilities](https://tmichoel.github.io/BioFindr.jl/dev/posteriorprobs/) of individial tests, an approach first proposed in [this paper](https://doi.org/10.1186/gb-2007-8-10-r219).


We will illustrate how to run causal inference with [BioFindr.jl][1] using [preprocessed data][2] from the [GEUVADIS study][3]. See the [installation instructions](installation.qmd) for the steps you need to take to reproduce this tutorial.

## Set up the environment


In [1]:
using DrWatson
quickactivate(@__DIR__)

using DataFrames
using Arrow

using BioFindr

## Load data

### Expression data

[BioFindr.jl][1] expects that expression data are stored as [floating-point numbers](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/) in a [DataFrame][4] where columns correspond to variables (genes) and rows to samples, see the [coexpression analysis tutorial](coexpression.qmd) for more details.

This tutorial uses two tables of expression data from the same set of samples, one for mRNA expression data called `dt`, and one for microRNA (miRNA) expression data called `dm`:


In [2]:
dt = DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dt.arrow")));
dm = DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dm.arrow")));

### Genotype data

[BioFindr.jl][1] expects that genotype data are stored as [integer numbers](https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers/) in a [DataFrame][4] where columns correspond to variables (genetic variants) and rows to samples. [BioFindr.jl][1] uses a categorical model to associate genetic variation to variation in gene expression, hence how different genotypes (e.g. heterozygous vs. homozygous) are encoded as integers does not matter. Future versions will support [scientific types](https://juliaai.github.io/ScientificTypes.jl/dev/) for representing genotype data.

This tutorial uses two tables of expression data from the same set of samples, one with genotypes for mRNA eQTLs called `dgt`, and one for microRNA (miRNA) eQTLs called `dgm`:


In [3]:
dgt = DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dgt.arrow")));
dgm = DataFrame(Arrow.Table(datadir("exp_pro","findr-data-geuvadis", "dgm.arrow")));

### eQTL mapping data

The [preprocessed GEUVADIS data][2] has been organized such that each column of the genotype data is the strongest eQTLs for the corresponding column in the matching expression data. Usually however, eQTL mapping data will be available in the form of a table with variant IDs, gene IDs, and various eQTL associaion statistics (see the [original GEUVADIS file](https://www.ebi.ac.uk/biostudies/files/E-GEUV-1/E-GEUV-1/analysis_results/EUR373.gene.cis.FDR5.all.rs137.txt.gz) for an example). [BioFindr.jl][1] expects that such a table is read into a [DataFrame][4], and that only the best (most strongly associated) eQTL is kept for each gene, that is, genes appear only once in the eQTL mapping table. Let's artificially create such tables for our data:


In [4]:
dpt = DataFrame(SNP_ID = names(dgt), GENE_ID=names(dt)[1:ncol(dgt)]);
dpm = DataFrame(SNP_ID = names(dgm), GENE_ID=names(dm)[1:ncol(dgm)]);

## Run BioFindr.jl

### Subset-to-all causal inference

In the most common scenario, we have a dataset of gene expression values, with significant cis-eQTL instruments for a subset of genes ("eGenes"). We can then infer causal interactions from the eGenes to all other genes (including other eGenes). For instance, to infer causal microRNA $\to$ microRNA interaction, run:


In [5]:
dP = findr(dm, dgm, dpm; FDR=0.1)

Row,Source,Target,Probability,qvalue
,String,String,Float64,Float64
1,hsa-miR-550a-2-5p,hsa-miR-550a-3-5p,1.0,0.0
2,hsa-miR-96-5p,hsa-miR-182-5p,1.0,4.81948e-13
3,hsa-miR-550a-2-5p,hsa-miR-550a-2-3p,1.0,2.53784e-11
4,hsa-miR-3130-1-5p,hsa-miR-3130-2-5p,1.0,1.04578e-10
5,hsa-miR-550a-2-3p,hsa-miR-550a-3-5p,1.0,1.5224e-10
6,hsa-miR-550a-2-3p,hsa-miR-550a-2-5p,1.0,4.87261e-10
7,hsa-miR-183-5p,hsa-miR-182-5p,1.0,5.45614e-9
8,hsa-miR-30a-3p,hsa-miR-30a-5p,1.0,4.29665e-8
9,hsa-miR-574-5p,hsa-miR-574-3p,0.999994,6.6706e-7


BioFindr computes a [posterior probability](https://tmichoel.github.io/BioFindr.jl/dev/posteriorprobs/) of a non-zero causal interaction for every pair pf **Source** and **Target** genes (columns of `dm`). The possible **Source** genes are the eGenes, the subset of genes with cis-eQTLs (columns of `dgm`), as defined by the eQTL-to-gene mapping in `dpm`.

By default the output is sorted by decreasing **Probability**. The optional parameter **FDR** can be used to limit the output to the set of pairs that has a [global false discovery rate (FDR)](https://en.wikipedia.org/wiki/False_discovery_rate#Storey-Tibshirani_procedure) less than a desired value (here set to 10%). The **qvalue** column in the output can be used for further filtering of the output, see the [coexpression analysis tutorial](coexpression.qmd) for more details.

Note the order of input arguments: first `dm`, the expression data, then `dgm`, the genotype data, and then `dpm`, the eQTL mapping of variants to eGenes.

By default, BioFindr assumes that the first column of `dpm` is a list of variant names that can be found in the column names of `dgm`, and that the second column of `dpm` is a list of gene names that can be found in the column names of `dm`:


In [6]:
dpm

Row,SNP_ID,GENE_ID
,String,String
1,rs201731283,hsa-miR-4804-5p
2,rs73933236,hsa-miR-641-5p
3,rs17269517,hsa-miR-2116-3p
4,rs2273626,hsa-miR-4707-3p
5,rs174559,hsa-miR-1908-5p
6,rs7095504,hsa-miR-2110-5p
7,rs2583391,hsa-miR-1255a-5p
8,rs5743580,hsa-miR-574-3p
9,rs143756085,hsa-miR-1270-2-5p


If your eQTL mapping DataFrame has the relevant columns in a different place, you need to use the optional arguments `colX` (for the gene expression IDs) and `colG` (for the eQTL genotype IDs) to specify either the relevant columns index or name:


In [7]:
dP = findr(dm, dgm, dpm; colX=2, colG=1, FDR=0.1);

or


In [8]:
dP = findr(dm, dgm, dpm; colX="GENE_ID", colG="SNP_ID", FDR=0.1);

Any other columns in the eQTL mapping DataFrame (such as association summary statistics) are ignored.

### Bipartite causal inference

In some applications, we have multiple omics datasets from the same samples, and may be interested in causal inference from one set of variables to another one. In the GEUVADIS study for instance, we may be interested in inferring causal interactions between microRNAs and target genes. To infer interactions *from* microRNAs *to* mRNAs, run:


In [9]:
dP = findr(dt, dm, dgm, dpm; FDR=0.1)

Row,Source,Target,Probability,qvalue
,String,String,Float64,Float64
1,hsa-miR-335-5p,ENSG00000128510.5,1.0,0.0
2,hsa-miR-335-3p,ENSG00000128510.5,1.0,0.0
3,hsa-miR-3150b-3p,ENSG00000245080.3,1.0,3.4931e-8
4,hsa-miR-345-5p,ENSG00000258504.1,1.0,1.39759e-7
5,hsa-miR-345-5p,ENSG00000197119.7,0.999992,1.70218e-6
6,hsa-let-7b-5p,ENSG00000241990.1,0.999987,3.63495e-6
7,hsa-miR-574-3p,ENSG00000197712.6,0.999967,7.86251e-6
8,hsa-miR-574-5p,ENSG00000197712.6,0.999965,1.12617e-5
9,hsa-let-7b-5p,ENSG00000197182.8,0.999911,1.99429e-5


### Causal test combinations

As indicated above, causal inference requires the combination of multiple statistical tests, and [BioFindr.jl][1] supports [multiple such combinations](https://tmichoel.github.io/BioFindr.jl/dev/testLLR/#Causal-inference), each with their own strengths and weaknesses. Comparisons between the different methods can be found in the [original BioFindr paper][5] and in a [follow-up paper][6]. In brief, the following combinations are available:

#### Instrumental variables (default)

The *instrumental variables* (IV) test combination infers a causal $A\to B$ interaction if there is a genetic association $E\to B$, where $E$ is the causal anchor (best cis-eQTL for $A$), and if $A$ and $B$ are not independently associated with $E$ (no pleiotropic effect). This test combination has been shown to provide a good balance between false negative and false positive rate and has a clear genetic interpretation. It is therefore recommended as the default combination. To use the instrumental variable test combination, you don't need to do anything (it's the default): 


In [10]:
dP = findr(dm, dgm, dpm; FDR=0.1);

or, you can explicitly set the optional parameter `combination="IV"`:


In [11]:
dP = findr(dm, dgm, dpm; FDR=0.1, combination="IV");

#### Mediation

The *mediation test* infers a causal $A\to B$ interaction if there is a genetic association $E\to B$ and if this association disappears after conditioning on $A$. If these two tests are true, the causal graph must be $E\to A\to B$. As a result, this test combination has a very low false positive rate, but in most applications it is hampered by a high false negative rate: if $A$ and $B$ are jointly regulated by a third factor $C$, the mediation test will fail even in the presence of a true $A\to B$ interaction. To use the mediation test combination, set the optional parameter `combination="mediation"`:


In [12]:
dP_med = findr(dm, dgm, dpm; FDR=0.1, combination="mediation");

#### Original BioFindr combination

In the [original BioFindr paper][5], a new test combination was proposed that combines the instrumental variable test combination with a new [relevance test](https://tmichoel.github.io/BioFindr.jl/dev/realLLR/#Relevance-test). On simulated data, this new combination further reduced the false negative rate compared to the IV combination. at the expense of an increased false positive rate, for an overall improved performance. However, the relevance test does not have a clear genetic or causal interpretation, and on real data, performance is generally equivalent to the IV combination, and therefore the IV combination is now recommended as the default. To use the original BioFindr test combination, set the optional parameter `combination="orig"`:


In [13]:
dP_orig = findr(dm, dgm, dpm; FDR=0.1, combination="orig");

[1]: https://github.com/tmichoel/BioFindr.jl
[2]: https://github.com/lingfeiwang/findr-data-geuvadis
[3]: https://doi.org/10.1038/nature12531
[4]: https://dataframes.juliadata.org/stable/
[5]: https://doi.org/10.1371/journal.pcbi.1005703
[6]: https://doi.org/10.1039/D0MO00140F